Account

In [3]:
# Loaded variable 'df' from URI: c:\Users\ohechihiro\Documents\d365-migration\cmt-files\formatted-data\account.parquet
import pandas as pd
df = pd.read_parquet(r'c:\Users\ohechihiro\Documents\d365-migration\cmt-files\formatted-data\account.parquet')

# Define a function to convert full-width to half-width
def to_half_width(text):
    if pd.isna(text):
        return text
    return text.translate(str.maketrans(
        "０１２３４５６７８９ＡＢＣＤＥＦＧＨＩＪＫＬＭＮＯＰＱＲＳＴＵＶＷＸＹＺａｂｃｄｅｆｇｈｉｊｋｌｍｎｏｐｑｒｓｔｕｖｗｘｙｚ　",
        "0123456789ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz "
    ))
# Apply the function to all object columns
df = df.applymap(lambda x: to_half_width(x) if isinstance(x, str) else x)



C:\Users\ohechihiro\AppData\Local\Temp\ipykernel_19360\196850196.py:14: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(lambda x: to_half_width(x) if isinstance(x, str) else x)


In [4]:
# Save the updated DataFrame back to the same Parquet file
df.to_parquet(r'c:\Users\ohechihiro\Documents\d365-migration\cmt-files\formatted-data\account.parquet', engine='pyarrow')
print("DataFrame has been saved to account.parquet.")

DataFrame has been saved to account.parquet.


In [17]:
import requests

# secret.txtを読み込む関数
def load_secrets(file_path):
    secrets = {}
    with open(file_path, "r") as file:
        for line in file:
            # 行を分割してキーと値を取得
            key, value = line.strip().split("=", 1)
            secrets[key] = value
    return secrets

# secret.txtのパス
secrets_file = "secrets.txt"

# 機密情報を読み込む
secrets = load_secrets(secrets_file)

# Azure Maps APIキー
API_KEY = secrets.get("AZURE_MAPS_API_KEY")

# ジオコーディング（住所から緯度・経度を取得）
def geocode_address(address):
    url = f"https://atlas.microsoft.com/search/address/json"
    params = {
        "api-version": "1.0",
        "subscription-key": API_KEY,
        "query": address
    }
    response = requests.get(url, params=params)
    data = response.json()
    
    if data.get("results"):
        location = data["results"][0]["position"]
        return {"latitude": location["lat"], "longitude": location["lon"]}
    else:
        return {"error": "Address not found"}

# 逆ジオコーディング（緯度・経度から住所を取得）
def reverse_geocode(lat, lon):
    url = f"https://atlas.microsoft.com/search/address/reverse/json"
    params = {
        "api-version": "1.0",
        "subscription-key": API_KEY,
        "query": f"{lat},{lon}"
    }
    response = requests.get(url, params=params)
    data = response.json()
    
    if data.get("addresses"):
        address = data["addresses"][0]["address"]["freeformAddress"]
        return address
    else:
        return {"error": "Location not found"}

# 使用例
address = "若宮1-35-2"
location = geocode_address(address)
print("Geocoded Location:", location)

latitude, longitude = 35.6785, 139.6823
address = reverse_geocode(latitude, longitude)
print("Reverse Geocoded Address:", address)

Geocoded Location: {'latitude': 42.264424, 'longitude': -85.889336}
Reverse Geocoded Address: 東京, Kanto
Reverse Geocoded Address: 東京, Kanto


In [ ]:
import requests

# secret.txtを読み込む関数
def load_secrets(file_path):
    secrets = {}
    with open(file_path, "r") as file:
        for line in file:
            # 行を分割してキーと値を取得
            key, value = line.strip().split("=", 1)
            secrets[key] = value
    return secrets

# secret.txtのパス
secrets_file = "secrets.txt"

# 機密情報を読み込む
secrets = load_secrets(secrets_file)

# Azure Maps APIキー
API_KEY = secrets.get("AZURE_MAPS_API_KEY")

# Send address1_composite to Azure Maps API and get latitude and longitude
def get_lat_lon(address):
    api_url = "https://atlas.microsoft.com/search/address/json"
    params = {
        "api-version": "1.0",
        "subscription-key": f"{API_KEY}",
        "query": address
    }
    response = requests.get(api_url, params=params)
    if response.status_code == 200:
        results = response.json().get("results", [])
        if results:
            position = results[0].get("position", {})
            return position.get("lat"), position.get("lon")
    return None, None
# Apply the function to the DataFrame
df[['address1_latitude', 'address1_longitude']] = df['address1_composite'].apply(
    lambda x: pd.Series(get_lat_lon(x))
)

In [25]:
import requests
import pandas as pd

# secret.txtを読み込む関数
def load_secrets(file_path):
    secrets = {}
    with open(file_path, "r") as file:
        for line in file:
            # 行を分割してキーと値を取得
            key, value = line.strip().split("=", 1)
            secrets[key] = value
    return secrets

# secret.txtのパス
secrets_file = "secrets.txt"

# 機密情報を読み込む
secrets = load_secrets(secrets_file)

# Azure Maps APIキー
API_KEY = secrets.get("AZURE_MAPS_API_KEY")

# 同期型でAzure Maps APIにリクエストを送信する関数
def fetch_lat_lon(address):
    if not address:  # addressが値を持たない場合はリクエストを送信しない
        return None, None

    url = "https://atlas.microsoft.com/search/address/json"
    params = {
        "api-version": "1.0",
        "subscription-key": API_KEY,
        "query": address
    }
    try:
        response = requests.get(url, params=params)
        if response.status_code == 200:
            data = response.json()
            if data.get("results"):
                location = data["results"][0]["position"]
                return location["lat"], location["lon"]
        return None, None
    except Exception as e:
        print(f"Error fetching data for address '{address}': {e}")
        return None, None

# DataFrameの住所列を同期型で処理
addresses = df['address1_composite'].tolist()
results = [fetch_lat_lon(address) for address in addresses]

# DataFrameに結果を追加
df[['address1_latitude', 'address1_longitude']] = pd.DataFrame(results, columns=['latitude', 'longitude'])
print("Latitude and Longitude have been added to the DataFrame.")

Latitude and Longitude have been added to the DataFrame.


In [8]:
import MeCab

# MeCab Taggerオブジェクトを作成
mecab = MeCab.Tagger()

# 簡単な日本語の文を分析
text = "私は日本語を勉強しています。"
result = mecab.parse(text)

print(result)

私	ワタクシ	ワタクシ	私-代名詞	代名詞			0
は	ワ	ハ	は	助詞-係助詞			
日本	ニッポン	ニッポン	日本	名詞-固有名詞-地名-国			3
語	ゴ	ゴ	語	名詞-普通名詞-一般			1
を	オ	ヲ	を	助詞-格助詞			
勉強	ベンキョー	ベンキョウ	勉強	名詞-普通名詞-サ変可能			0
し	シ	スル	為る	動詞-非自立可能	サ行変格	連用形-一般	0
て	テ	テ	て	助詞-接続助詞			
い	イ	イル	居る	動詞-非自立可能	上一段-ア行	連用形-一般	0
ます	マス	マス	ます	助動詞	助動詞-マス	終止形-一般	
。			。	補助記号-句点			
EOS



In [136]:
import MeCab
import openai
from foundry_local import FoundryLocalManager

# --- MeCab初期化（人名辞書を組み込んだdicdirを指定） ---
# 例: mecab-ipadic-NEologd を利用する場合
mecab = MeCab.Tagger()

# By using an alias, the most suitable model will be downloaded
# to your end-user's device.
alias = "Phi-3.5-mini-instruct-cuda-gpu:1"

# Create a FoundryLocalManager instance. This will start the Foundry
# Local service if it is not already running and load the specified model.
manager = FoundryLocalManager(alias)

# The remaining code uses the OpenAI Python SDK to interact with the local model.

# Configure the client to use the local Foundry service
client = openai.OpenAI(
    base_url=manager.endpoint,
    api_key=manager.api_key  # API key is not required for local usage
)

def kana_with_mecab(name: str):
    """MeCabで人名のカナを取得（既知名のみ）"""
    node = mecab.parseToNode(name)
    results = []
    while node:
        surface = node.surface
        features = node.feature.split(",")
        if len(features) > 7 and features[0] == "名詞" and features[1] == "固有名詞" and features[2] == "人名":
            yomi = features[7] if features[7] != "*" else None
            results.append((surface, yomi))
        node = node.next
    return results

def kana_with_foundry(name: str):
    """Foundry Local APIで未知名のカナを補完"""
    try:
        resp = client.chat.completions.create(
            model=manager.get_model_info(alias).id,
            messages=[
                {"role": "system", "content": "あなたは日本語の人名にカタカナの読みを振るアシスタントです。"},
                {"role": "user", "content": f" {name}の読みを返します：カタカナ"}
            ],
        )
        return resp.choices[0].message.content.strip()
    except Exception as e:
        print(f"Error in kana_with_foundry: {e}")
        return None

def normalize_name(name: str):
    """ハイブリッド処理：MeCabで既知名、未知名はFoundry Local"""
    mecab_results = kana_with_mecab(name)
    final_results = []
    result = [yomi for surface, yomi in mecab_results if yomi]
    if result:
        final_results.append(result)
    else:
        # MeCabで全く解析できなかった場合、名前全体をFoundryに投げる
        ai_yomi = kana_with_foundry(name)
        final_results.append(ai_yomi)

    return final_results

# --- 実行例 ---
names = ["虎豚", "大江美奈子", "Michael","葵", "牛丼太郎","マイケルジャクソン"]
for n in names:
    result = normalize_name(n)
    print(n, "→", result)


虎豚 → ['トントゥ']
大江美奈子 → [['オオエ', 'ミナコ']]
Michael → ['ミカロー']
葵 → ['オキ']
牛丼太郎 → ['牛丼 タロウ']
マイケルジャクソン → [['マイケル-Michael', 'ジャクソン-Jackson']]


In [39]:
from foundry_local import FoundryLocalManager

# Initialize and optionally bootstrap with a model
manager = FoundryLocalManager(alias_or_model_id=None, bootstrap=True)

In [132]:
from openai import OpenAI

client = OpenAI(
    base_url = "http://localhost:5272/v1/",
    api_key = "unused", # required for the API but not used
)

# Set the model to use and generate a streaming response
stream = client.chat.completions.create(
    messages=[{"role": "system", "content": "あなたは日本語の人名にカタカナの読みを振るアシスタントです。"},
            {"role": "user", "content": "次の名前にカタカナを振ってください: 葵"},],
    model = "Phi-3.5-mini-instruct-cuda-gpu:1"
)

# Print the streaming response
print(stream.choices[0].message.content)

 葵 (あおい)

カタカナ表記では「アオイ」となります。これは、日本の名前に使われることが多い漢字で、特に女性の名前に使われることが多いです。
